#### Загрузка необходимых данных и библиотек в рабочую среду

In [1]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

In [2]:
ratings = pd.read_csv('ratings.csv')

#### Перевод pandas датафрейма в формат surprise

In [3]:
# Создаем объект Reader
reader = Reader(rating_scale=(0.5, 5))

In [4]:
# Загружаем данные в формате Surprise
data_surprise = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [5]:
# Разделяем данные на train и test
trainset, testset = train_test_split(data_surprise, test_size=.2)

#### Обучение моделей

In [6]:
# Обучаем модель SVD с подобранными в прошлой работе наилучшими параметрами
svd = SVD(n_factors=350, n_epochs=70, lr_all=0.007, reg_all=0.09, random_state=42)
svd.fit(trainset)

In [7]:
# Обучаем модель ALS с параметрами по умолчанию и фиксированной случайностью
from surprise import NMF
als = NMF(random_state=42)
als.fit(trainset)

#### Реализация гибридной рекомендательной системы

In [8]:
# Вычисляем взвешенную оценку фильмов для каждого пользователя
def get_weighted_ratings(user_id, trainset, model1, model2, weights=[0.7, 0.3]):
    """Вычисляет взвешенную оценку фильмов для заданного пользователя.

    Args:
        user_id (int): Идентификатор пользователя.
        trainset (surprise.Trainset): Тренировочный набор данных.
        model1 (surprise.AlgoBase): Первая модель.
        model2 (surprise.AlgoBase): Вторая модель.
        weights (list): Веса для каждой модели.

    Returns:
        dict: Словарь с оценками фильмов для данного пользователя.
    """
    user_ratings = trainset.ur[user_id]  # Получаем оценки пользователя
    ratings = {}
    for movie_id in trainset.all_items():  # Проходим по всем фильмам
        if movie_id not in user_ratings:  # Если пользователь не оценил фильм
            rating1 = model1.predict(user_id, movie_id).est  # Получаем оценку первой модели
            rating2 = model2.predict(user_id, movie_id).est  # Получаем оценку второй модели
            weighted_rating = weights[0] * rating1 + weights[1] * rating2  # Вычисляем взвешенную оценку
            ratings[movie_id] = weighted_rating  # Добавляем оценку в словарь
    return ratings

In [9]:
# Загружаем названия фильмов
movies = pd.read_csv('movies.csv')

In [10]:
# Выводим топ-5 рекомендуемых фильмов для заданного пользователя
user_id = 42
ratings = get_weighted_ratings(user_id, trainset, svd, als)
top_5_movies = sorted(ratings.items(), key=lambda x: x[1], reverse=True)[:5]
for movie_id, rating in top_5_movies:
    movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
    print(f'{movie_id} ({movie_title}): {rating:.2f}')

3502 (My Life (1993)): 4.73
3200 (Last Detail, The (1973)): 4.72
550 (Threesome (1994)): 4.68
2905 (Sanjuro (Tsubaki Sanjûrô) (1962)): 4.67
213 (Burnt by the Sun (Utomlyonnye solntsem) (1994)): 4.65
